In [1]:
import os, re, time, json, math
import datetime
import PIL.Image, PIL.ImageFont, PIL.ImageDraw
import numpy as np
import tensorflow as tf
import google
from matplotlib import pyplot as plt
print("Tensorflow version " + tf.__version__)

Tensorflow version 2.3.1


In [2]:
import tensorflow as tf
import tensorflow.keras as keras

from src.generators.tf_records_generator import get_dataset, count_data_items
from src.models.efficientNet_B0 import EffNet0, freeze_blocks
from src.utils.log_files import save_config, save_fold_iter_history, get_log_dir
from src.utils.kfold import get_kfold_split

In [3]:
CONFIG = dict(
    effnet_version = 0,
    input_shape=(256, 256, 3),
    image_resolution=256,
    trainable_base=False,
    time=datetime.datetime.now().strftime("_%d_%m_%Y_%H_%M"),
    use_patient_data=True,
    inner_blocks_frozen=4,

    replicas=1,
    steps_per_epoch=None,
    validation_steps=None,
    batch_size=8,
    epochs=5,
    optimizer='adam',
    loss='binary_crossentropy',
    
    output_bias=np.log([584/32542]),
    weight_for_0 = (1 / 32542)*(32542+584)/2.0,
    weight_for_1 = (1 / 584)*(32542+584)/2.0
)

In [4]:
IS_COLAB_BACKEND = 'COLAB_GPU' in os.environ  # this is always set on Colab, the value is 0 or 1 depending on GPU presence
if IS_COLAB_BACKEND:
  from google.colab import auth
  # Authenticates the Colab machine and also the TPU using your
  # credentials so that they can access your private GCS buckets.
  auth.authenticate_user()
  data_dir = 'gs://dataset_files/'
else:
  data_dir = 'dataset/'

In [5]:
# Detect hardware
try:
  tpu_resolver = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError:
  tpu_resolver = None
  gpus = tf.config.experimental.list_logical_devices("GPU")

# Select appropriate distribution strategy
if tpu_resolver:
  tf.config.experimental_connect_to_cluster(tpu_resolver)
  tf.tpu.experimental.initialize_tpu_system(tpu_resolver)
  strategy = tf.distribute.TPUStrategy(tpu_resolver)
  print('Running on TPU ', tpu_resolver.cluster_spec().as_dict()['worker'])
elif len(gpus) > 1:
  strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
  print('Running on multiple GPUs ', [gpu.name for gpu in gpus])
elif len(gpus) == 1:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on single GPU ', gpus[0].name)
else:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on CPU')
  
print("Number of accelerators: ", strategy.num_replicas_in_sync)
CONFIG["replicas"] = strategy.num_replicas_in_sync


Running on single GPU  /device:GPU:0
Number of accelerators:  1


In [6]:
path = data_dir + str(CONFIG["image_resolution"])
files_train = np.sort(np.array(tf.io.gfile.glob(path + '/train*.tfrec')))
files_test = np.sort(np.array(tf.io.gfile.glob(path + '/test*.tfrec')))


ds = get_dataset(files_train[:-3], CONFIG)
val_ds = get_dataset(files_train[-3], CONFIG)
test_ds = get_dataset(files_test, CONFIG)

In [13]:
def make_model(config:dict, strategy)    :
    with strategy.scope():
      model = EffNet0(config=config, trainable_base=config['trainable_base'])
      #model.layers[9].trainable = False
      #model.layers[9] = unfreeze_blocks(model.layers[9],4)
      #model.layers[9].layers[200].trainable = True
      model.compile(optimizer=config["optimizer"], loss=config["loss"], 
                    metrics=[
                                  keras.metrics.TruePositives(name='tp'),
                                  keras.metrics.FalsePositives(name='fp'),
                                  keras.metrics.TrueNegatives(name='tn'),
                                  keras.metrics.FalseNegatives(name='fn'), 
                                  keras.metrics.BinaryAccuracy(name='accuracy'),
                                  keras.metrics.Precision(name='precision'),
                                  keras.metrics.Recall(name='recall'),
                                  keras.metrics.AUC(name='auc'),
                            ]
    )
    return model


In [14]:
kfold_split = get_kfold_split(files_train)
save_config(CONFIG)
for iter, files in kfold_split.items():
    train_ds = get_dataset(files['train'],CONFIG, patient_info=True,repeat=True) 
    val_ds = get_dataset(files['validation'], CONFIG, patient_info=True,repeat=True)
    
    steps_per_epoch = math.ceil(count_data_items(files["train"])/(CONFIG["batch_size"]*CONFIG["replicas"]))
    validation_steps = math.ceil(count_data_items(files["validation"])/(CONFIG["batch_size"]*CONFIG["replicas"]))
    
    model = make_model(CONFIG, strategy=strategy)    
    
    weights = model.layers[9].layers[200].weights[0][0][0][0][0]
    poczatek = model.layers[9].layers[4].weights[0][0][0][0][0]
    
    history = model.fit(train_ds,class_weight={0:CONFIG["weight_for_0"],1:CONFIG["weight_for_1"]},
                        validation_data=val_ds,validation_batch_size=CONFIG["batch_size"]*CONFIG["replicas"],
                        batch_size=CONFIG["batch_size"]*CONFIG["replicas"], epochs=CONFIG["epochs"],
                        steps_per_epoch=20,validation_steps=20)
    
    weights2 = model.layers[9].layers[200].weights[0][0][0][0][0]
    poczatek2 = model.layers[9].layers[4].weights[0][0][0][0][0]
    if np.all(weights == weights2):
        print("udalo sie zamrozic")
    else:
        print("kicha")
    if np.all(poczatek == poczatek2):
        print("poczatek zamrozony")
    else:
        print("poczatek kicha")
    
    save_fold_iter_history(CONFIG, history.history,iter)
    



Epoch 1/5
20/20 [==============================] - 6s 276ms/step - loss: 3.3910 - tp: 0.0000e+00 - fp: 9.0000 - tn: 148.0000 - fn: 3.0000 - accuracy: 0.9250 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.2102 - val_loss: 0.2531 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 157.0000 - val_fn: 3.0000 - val_accuracy: 0.9812 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5000

In [15]:
log_dir = get_log_dir(CONFIG)
zip_name = os.path.basename(log_dir)
!zip -r $zip_name $log_dir

  adding: logs/TL_EffN0_256_02_11_2020_21_13/ (stored 0%)
  adding: logs/TL_EffN0_256_02_11_2020_21_13/0.json (deflated 52%)
  adding: logs/TL_EffN0_256_02_11_2020_21_13/config.json (deflated 33%)
  adding: logs/TL_EffN0_256_02_11_2020_21_13/1.json (deflated 53%)


In [11]:
model = make_model(CONFIG, strategy=strategy)

In [12]:
model.summary()

Model: "functional_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           [(None, 9)]          0                                            
__________________________________________________________________________________________________
dense_18 (Dense)                (None, 256)          2560        input_19[0][0]                   
__________________________________________________________________________________________________
batch_normalization_18 (BatchNo (None, 256)          1024        dense_18[0][0]                   
__________________________________________________________________________________________________
activation_18 (Activation)      (None, 256)          0           batch_normalization_18[0][0]     
______________________________________________________________________________________

In [32]:
model.layers[9].layers[50].count_params()

3600

In [ ]:
l = model.layers[-1].weights[0][0]
a = model.layers[-1].weights[0][0] +1

In [58]:
l = model.layers[9].layers[200].weights[0][0][0][0][0]
a = model.layers[9].layers[200].weights[0][0][0][0][0] +1

In [12]:
model.layers[9].layers[2]

In [ ]:
np.all(l == model.layers[-1].weights[0])

In [10]:
model.layers[9].summary()

Model: "efficientnetb0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 256, 256, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 256, 256, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 257, 257, 3)  0           normalization[0][0]              
_____________________________________________________________________________________

In [13]:
model.layers[9] = unfreeze_blocks(model.layers[9],4)

['block7', 'block6', 'block5', 'block4', 'top_']
input_6
rescaling_1
normalization_1
stem_conv_pad
stem_conv
stem_bn
stem_activation
block1a_dwconv
block1a_bn
block1a_activation
block1a_se_squeeze
block1a_se_reshape
block1a_se_reduce
block1a_se_expand
block1a_se_excite
block1a_project_conv
block1a_project_bn
block2a_expand_conv
block2a_expand_bn
block2a_expand_activation
block2a_dwconv_pad
block2a_dwconv
block2a_bn
block2a_activation
block2a_se_squeeze
block2a_se_reshape
block2a_se_reduce
block2a_se_expand
block2a_se_excite
block2a_project_conv
block2a_project_bn
block2b_expand_conv
block2b_expand_bn
block2b_expand_activation
block2b_dwconv
block2b_bn
block2b_activation
block2b_se_squeeze
block2b_se_reshape
block2b_se_reduce
block2b_se_expand
block2b_se_excite
block2b_project_conv
block2b_project_bn
block2b_drop
block2b_add
block3a_expand_conv
block3a_expand_bn
block3a_expand_activation
block3a_dwconv_pad
block3a_dwconv
block3a_bn
block3a_activation
block3a_se_squeeze
block3a_se_reshap

In [39]:
model.layers[9].layers[200].trainable

True

In [43]:
for layer in model.layers[9].layers:
    print(f"{layer.name}\t\t : {layer.trainable}\n")

input_60		 : True

rescaling_19		 : True

normalization_19		 : False

stem_conv_pad		 : False

stem_conv		 : False

stem_bn		 : False

stem_activation		 : False

block1a_dwconv		 : False

block1a_bn		 : False

block1a_activation		 : False

block1a_se_squeeze		 : False

block1a_se_reshape		 : False

block1a_se_reduce		 : False

block1a_se_expand		 : False

block1a_se_excite		 : False

block1a_project_conv		 : False

block1a_project_bn		 : False

block2a_expand_conv		 : False

block2a_expand_bn		 : False

block2a_expand_activation		 : False

block2a_dwconv_pad		 : False

block2a_dwconv		 : False

block2a_bn		 : False

block2a_activation		 : False

block2a_se_squeeze		 : False

block2a_se_reshape		 : False

block2a_se_reduce		 : False

block2a_se_expand		 : False

block2a_se_excite		 : False

block2a_project_conv		 : False

block2a_project_bn		 : False

block2b_expand_conv		 : False

block2b_expand_bn		 : False

block2b_expand_activation		 : False

block2b_dwconv		 : False

block2b_bn		 :

In [37]:
model.summary()

Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           [(None, 9)]          0                                            
__________________________________________________________________________________________________
dense_14 (Dense)                (None, 256)          2560        input_16[0][0]                   
__________________________________________________________________________________________________
batch_normalization_14 (BatchNo (None, 256)          1024        dense_14[0][0]                   
__________________________________________________________________________________________________
activation_14 (Activation)      (None, 256)          0           batch_normalization_14[0][0]     
_______________________________________________________________________________________

In [16]:
from src.models.efficientNet_B0 import freeze_blocks

ImportError: cannot import name 'freeze_blocks' from 'src.models.efficientNet_B0' (D:\inzynierka\src\models\efficientNet_B0.py)